# Aprendizado Federado para reconhecimento de contexto em dispositivos móveis
## Experimento 1 - Pré-treinamento usual com uma base de usuários e Aprendizado Federado embarcado partindo do modelo pretreinado 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import es_utils as utils

Inicialmente precisamos separar quais 'labels' usaremos. Aqui, optamos pelas 51 labels que segundo artigo do ExtraSensosy utiliza, de modo a ter bons parâmetros de comparação

In [2]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

"labels = ['label:LYING_DOWN',\n 'label:SITTING',\n 'label:FIX_walking',\n 'label:FIX_running',\n 'label:BICYCLING',\n 'label:SLEEPING',\n 'label:LAB_WORK',\n 'label:IN_CLASS',\n 'label:IN_A_MEETING',\n 'label:LOC_main_workplace',\n 'label:OR_indoors',\n 'label:OR_outside',\n 'label:IN_A_CAR',\n 'label:ON_A_BUS',\n 'label:DRIVE_-_I_M_THE_DRIVER',\n 'label:DRIVE_-_I_M_A_PASSENGER',\n 'label:LOC_home',\n 'label:FIX_restaurant',\n 'label:PHONE_IN_POCKET',\n 'label:OR_exercise',\n 'label:COOKING',\n 'label:SHOPPING',\n 'label:STROLLING',\n 'label:DRINKING__ALCOHOL_',\n 'label:BATHING_-_SHOWER',\n 'label:CLEANING',\n 'label:DOING_LAUNDRY',\n 'label:WASHING_DISHES',\n 'label:WATCHING_TV',\n 'label:SURFING_THE_INTERNET',\n 'label:AT_A_PARTY',\n 'label:AT_A_BAR',\n 'label:LOC_beach',\n 'label:SINGING',\n 'label:TALKING',\n 'label:COMPUTER_WORK',\n 'label:EATING',\n 'label:TOILET',\n 'label:GROOMING',\n 'label:DRESSING',\n 'label:AT_THE_GYM',\n 'label:STAIRS_-_GOING_UP',\n 'label:STAIRS_-_GOING

In [3]:
labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]

len(labels)

6

Separamos os 60 usuários em 5 pastas onde cada pasta considera 40 usuários selecionados aleatoriamente para treino e os 20 restantes para teste. Os dados dos 20 usuários são ainda separados em 4 arquivos *.csv* (x_train, x_test, y_train, y_test) e colocados em pastas nomeadas com o uuid do usuário.

In [4]:
#k_folds = utils.create_k_folds_n_users(5, 3, '/home/wander/OtherProjects/har_flower/sample_data')
k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

Em seguida, para cada pasta, treinamos um modelo de MLP fazendo uma otimização de hiperparâmetros e convertemos em um modelo *tflite*. Os hiperparâmetros otimizados são:

- número de camadas escondidas (*hidden layers*): 1 ou 2
- número de neurônios em cada camada escondida: 4 a 64, com um passo de tamanho 4
- constante de penalização da regularização L2 em cada camada: 1e-2, 1e-3 ou 1e-4
- taxa de aprendizado (*learning rate*) do otimizador (*Stochastic Gradient Descent*, **SGD**): 1e-1, 1e-2, 1e-3 ou 1e-4
- momentum do otimizador (**SGD**): 0.8, 0.5, 0.3 ou 0.1

In [ ]:

for fold in k_folds:
    config = {
        'df_path': k_folds[fold]['40'],
        'labels': labels
    }
    
    har = utils.HAR(config)
    #har.run()
    har.hypertunning()
    
    har.mlp.model.save(f'model/saved_model_{fold}')
    converter = tf.lite.TFLiteConverter.from_saved_model(f'model/saved_model_{fold}') # path to the SavedModel directory
    tflite_model = converter.convert()
    with open(f'model/model_base_{fold}.tflite', 'wb') as f:
      f.write(tflite_model)

Trial 30 Complete [00h 00m 20s]
val_categorical_accuracy: 0.38854455947875977

Best val_categorical_accuracy So Far: 0.38854455947875977
Total elapsed time: 00h 07m 03s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
3266/3266 [==============================] - 3s 823us/step - loss: 117761056768.0000 - categorical_accuracy: 0.3719 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 2/100
3266/3266 [==============================] - 3s 769us/step - loss: 0.7863 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 3/100
3266/3266 [==============================] - 3s 766us/step - loss: 0.7863 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 4/100
3266/3266 [==============================] - 3s 774us/step - loss: 0.7863 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 5/100
3266/3266 [==============================] - 3s 771us/step - loss: 0.7863 - categorical_accuracy: 

3266/3266 [==============================] - 2s 762us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 49/100
3266/3266 [==============================] - 2s 763us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 50/100
3266/3266 [==============================] - 2s 764us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 51/100
3266/3266 [==============================] - 3s 769us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 52/100
3266/3266 [==============================] - 2s 764us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 53/100
3266/3266 [==============================] - 3s 773us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885

3266/3266 [==============================] - 2s 763us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 97/100
3266/3266 [==============================] - 2s 760us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 98/100
3266/3266 [==============================] - 3s 773us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 99/100
3266/3266 [==============================] - 2s 764us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Epoch 100/100
3266/3266 [==============================] - 2s 765us/step - loss: 0.7862 - categorical_accuracy: 0.3720 - val_loss: nan - val_categorical_accuracy: 0.3885
Best epoch: 1
1472/1472 [==============================] - 1s 507us/step - loss: nan - categorical_accuracy: 0.3696
Test results - Loss: nan - Accuracy: 0.369559228420

2022-03-22 13:33:34.298464: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/saved_model_fold_0/assets


2022-03-22 13:33:34.755651: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-22 13:33:34.755669: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-22 13:33:34.755673: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-22 13:33:34.756277: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model_fold_0
2022-03-22 13:33:34.757777: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-22 13:33:34.757789: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model_fold_0
2022-03-22 13:33:34.761404: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-22 13:33:34.799181: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: mod

Com os modelos salvos, o restante do experimento segue em ambiente Android

In [ ]:
har.mlp.model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skmultilearn.model_selection import iterative_train_test_split



raw = k_folds['fold_0']['train']
x =raw[raw.columns.drop(raw.filter(regex='label:'))]
y = raw.filter(regex='label:')
#x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, y_train, x_test, y_test = iterative_train_test_split(x, y, test_size = 0.1)
#raw

In [ ]:
y_pred = har.mlp.model.predict(x_test)

In [ ]:
def avg_multilabel_BA(y_truei, y_predi):
    ba_array = []
    y_true, y_pred = remove_nans_np(y_truei, y_predi)
    #print(y_pred.shape)
    print(len(y_true))
    print(len(y_truei))
    for i in range(y_pred.shape[1]):
        report = classification_report(y_true.to_numpy()[:, i], (y_pred[:, i] > 0.5), output_dict=True, zero_division=0)
        #sensitivity = report['1.0']['recall'] # tp / (tp + fn)
        specificity = report['0.0']['recall'] #specificity = tn / (tn+fp)
        try:
            sensitivity = report['1.0']['recall'] # tp / (tp + fn)
        except:
            sensitivity = specificity # tp / (tp + fn)
        ba = 0.5*(specificity+sensitivity)
        ba_array.append(ba)
    return np.mean(ba_array)


def remove_nans_np(x, y):
    mask = ~np.isnan(x) & ~np.isnan(y)
    #mask = math.logical_and(math.logical_not(math.is_nan(x)), math.logical_not(math.is_nan(y)))
    
    marra = np.ma.MaskedArray(x, mask=~mask)
    marrb = np.ma.MaskedArray(y, mask=~mask)

    return np.ma.compressed(marra), np.ma.compressed(marrb)

In [ ]:
avg_multilabel_BA(y_test, y_pred)

In [ ]:
raw[raw.columns.drop(raw.filter(regex='label:'))].info()

In [ ]:
raw.isna().sum()

In [ ]:
raw.filter(regex='label:')['label:SITTING'].loc [raw.filter(regex='label:')['label:SITTING'] == 1]

raw = pd.read_csv('../input/user1.features_labels.csv')
har = utils.HAR(config)
har.run()
#har.hypertunning()